In [1]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.analysis.make_analysis_data as ad
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

## Read data

In [2]:
df = hd.read_raw_sample("777")
hd.inspect(df)

Time for read_raw_sample               : 4.75 seconds
(682,656, 27)


,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,688293,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,"9572 24jan12 , tcs bowdon , bowdon gb - pos",Debit,25.030001,No Tag,No Tag,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-24,U
1,688291,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,warrington b.c. - d/d,Debit,26.000000,Gym Membership,Gym Membership,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-23,U


## Pipeline

In [3]:
clean = md.clean_data(df)

Time for rename_cols                   : 0.04 seconds
Time for clean_headers                 : 0.00 seconds
Time for drop_unneeded_vars            : 0.01 seconds
Time for add_year_month_variable       : 0.04 seconds
Time for drop_first_and_last_month     : 0.07 seconds
Time for lowercase_categories          : 1.00 seconds
Time for drop_missing_txn_desc         : 0.05 seconds
Time for gender_to_female              : 0.03 seconds
Time for credit_debit_to_debit         : 0.01 seconds
Time for sign_amount                   : 0.00 seconds
Time for missings_to_nan               : 0.01 seconds
Time for zero_balances_to_missing      : 0.00 seconds
Time for add_tag                       : 0.36 seconds
Time for tag_corrections               : 0.55 seconds
Time for add_tag_group                 : 0.12 seconds
Time for drop_duplicates               : 0.31 seconds
Time for order_and_sort                : 0.18 seconds
Time for clean_data                    : 2.86 seconds


## Dev

In [30]:
data = cl.rename_cols(df).pipe(cl.clean_headers)

Time for rename_cols                   : 0.04 seconds
Time for clean_headers                 : 0.00 seconds


Benchmark

In [31]:
dd = data.copy()
bm = cl.lowercase_categories(dd)

Time for lowercase_categories          : 1.09 seconds


New

In [69]:
k = pd.DataFrame({"data": ["HO", "ho", "ha"]})
k["data"] = k.data.astype("category")
k

,data
0,HO
1,ho
2,ha


In [89]:
[x.islower() for x in ["Akb adf", "baz ho", "Z", "ho"]]

[False, True, False, True]

In [90]:
def new(df):
    cat_vars = df.select_dtypes("category").columns
    for col in cat_vars:
        categories = df[col].cat.categories
        for c in categories:
            if not c.islower() and c.lower() in categories:
                df.loc[df[col] == c] = c.lower()
        df[col] = df[col].cat.remove_unused_categories()
    return cat_vars


dd = data.copy()
new(dd)

ValueError: Cannot setitem on a Categorical with a new category, set the categories first

In [ ]:
%debug

> /Users/fgu/miniconda3/envs/entropy/lib/python3.9/site-packages/pandas/core/arrays/categorical.py(2044)_validate_setitem_value()
   2042         # something to np.nan
   2043         if len(to_add) and not isna(to_add).all():
-> 2044             raise ValueError(
   2045                 "Cannot setitem on a Categorical with a new "
   2046                 "category, set the categories first"



ipdb>  to_add


Index(['100026'], dtype='object')


In [51]:
"a".islower()

True

What I want:

- When e.lower() already in others, then replace with duplicate from others



In [24]:
data

,id,user_id,user_registration_date,yob,salary_range,postcode,lsoa,msoa,gender,date,account_id,account_provider,account_type,latest_balance,desc,credit_debit,amount,tag_up,tag_manual,tag_auto,merchant,merchant_business_line,account_created,account_last_refreshed,data_warehouse_date_created,data_warehouse_date_last_updated,updated_flag
0,688293,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-25,262916,natwest bank,current,364.220001,"9572 24jan12 , tcs bowdon , bowdon gb - pos",debit,25.030001,no tag,no tag,no tag,no merchant,unknown merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-24,u
1,688291,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-25,262916,natwest bank,current,364.220001,warrington b.c. - d/d,debit,26.000000,gym membership,gym membership,no tag,no merchant,unknown merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-23,u
2,688290,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-25,262916,natwest bank,current,364.220001,"9572 24jan12 , apple itunes store, gbp , london gb - pos",debit,6.970000,mobile app,no tag,"entertainment, tv, media",apple,apple,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-08-15,u
3,688292,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-25,262916,natwest bank,current,364.220001,<mdbremoved>,credit,1901.329956,salary or wages (main),salary or wages (main),salary or wages (main),no merchant,no merchant business line,2011-07-20,2020-07-21 20:32:00,2014-07-18,2020-05-19,u
4,688294,777,2011-07-20,1969.0,20k to 30k,wa1 4,e01012553,e02002603,m,2012-01-30,262916,natwest bank,current,364.220001,legal & gen mi c/l - d/d,debit,16.190001,life insurance,no tag,insurance,legal & general,legal & general,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-11-13,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682651,807020573,576777,2020-03-11,1995.0,10k to 20k,dn5 9,e01007514,e02001551,m,2020-08-07,1634983,hsbc,current,1068.420044,cash barclay <mdbremoved>,debit,20.000000,cash,no tag,cash,no merchant,personal,2020-03-11,2020-08-14 14:50:00,2020-08-15,1900-01-01,c
682652,807020571,576777,2020-03-11,1995.0,10k to 20k,dn5 9,e01007514,e02001551,m,2020-08-07,1634983,hsbc,current,1068.420044,paypal *wetherspooxxxxxxxx001,debit,3.750000,dining or going out,no tag,dining or going out,wetherspoon,wetherspoon,2020-03-11,2020-08-14 14:50:00,2020-08-15,1900-01-01,c
682653,807020572,576777,2020-03-11,1995.0,10k to 20k,dn5 9,e01007514,e02001551,m,2020-08-07,1634983,hsbc,current,1068.420044,amznmktplace amazon.co.uk,debit,28.980000,enjoyment,no tag,enjoyment,amazon,amazon,2020-03-11,2020-08-14 14:50:00,2020-08-15,1900-01-01,c
682654,807020569,576777,2020-03-11,1995.0,10k to 20k,dn5 9,e01007514,e02001551,m,2020-08-07,1634983,hsbc,current,1068.420044,asda store 4519 south elmsall,debit,11.150000,"food, groceries, household",no tag,"food, groceries, household",asda,asda supermarket,2020-03-11,2020-08-14 14:50:00,2020-08-15,1900-01-01,c
